In [ ]:
from google.colab import drive
import json
import pickle
import math
import numpy as np
import pandas as pd
import spacy
from collections import Counter
import torch
import torchtext
import torch.nn as nn
import torch.optim as optim
from torch import Tensor
import random
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from torchtext.vocab import vocab
from torch.nn import TransformerEncoder, TransformerDecoder, TransformerEncoderLayer, TransformerDecoderLayer
import io
import time

In [ ]:
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
# Read JSON data from file
with open('/gdrive/MyDrive/train_data2.json', 'r') as file:
    data = json.load(file)

In [ ]:
# Read JSON data from file
with open('/gdrive/MyDrive/Test_data2_final.json', 'r') as file:
    val_data = json.load(file)

In [ ]:
import string

# Define the remove_punctuations function
def remove_punctuations(sentence):
    punctuations = list(string.punctuation)
    cleaned = ""
    for letter in sentence:
        if letter not in punctuations:
            cleaned += letter
    return cleaned

In [ ]:
# Process JSON data
source_sentences_train = []
target_sentences_train = []
len_malyalam = []
len_english = []

id_train = []

for language_pair, language_data in data.items():
    if(language_pair == "English-Malayalam"):
      print(f"Language Pair: {language_pair}")
      for data_type, data_entries in language_data.items():
          print(f"  Data Type: {data_type}")
          for entry_id, entry_data in data_entries.items():
              source = entry_data["source"]
              target = entry_data["target"].lower()
              source = remove_punctuations(source)
              target = remove_punctuations(target)
              source_sentences_train.append(source)
              target_sentences_train.append(target)
              id_train.append(entry_id)
              len_malyalam.append(len(source.split(' ')))
              len_english.append(len(target.split(' ')))

Language Pair: English-Malayalam
  Data Type: Train


In [ ]:
test_source = []
valid_id = []
len_val=[]


for language_pair, language_data in val_data.items():
    if(language_pair == "English-Malayalam"):
      print(f"Language Pair: {language_pair}")
      for data_type, data_entries in language_data.items():
          print(f"  Data Type: {data_type}")
          for entry_id, entry_data in data_entries.items():
              source = entry_data["source"]
              source = remove_punctuations(source)
              test_source.append(source)
              valid_id.append(entry_id)
              len_val.append(len(source.split(' ')))

Language Pair: English-Malayalam
  Data Type: Test


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Calculate the maximum length of malyalam and English sentences
max_length_malyalam = max(len_malyalam)
max_length_english = max(len_english)

print(f"Maximum Length of malyalam Sentence: {max_length_malyalam} words")
print(f"Maximum Length of English Sentence: {max_length_english} words")

Maximum Length of malyalam Sentence: 82 words
Maximum Length of English Sentence: 114 words


In [ ]:
pip install indic-nlp-library

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 17.4 MB/s eta 0:00:00


In [ ]:
from indicnlp.tokenize import indic_tokenize
# Tokenize malyalam sentences
tokenized_source_sentences = []
for sentence in source_sentences_train:
    tokens = indic_tokenize.trivial_tokenize(sentence)
    tokenized_source_sentences.append(tokens)

In [ ]:
tok_val=[]
for sentence in test_source:
    tokens = indic_tokenize.trivial_tokenize(sentence)
    tok_val.append(tokens)

In [ ]:
# Tokenize English sentences
nlp_en = spacy.load('en_core_web_sm')
tokenized_target_sentences = []
for sentence in target_sentences_train:
    doc = nlp_en(sentence)
    tokens = [token.text for token in doc]
    tokenized_target_sentences.append(tokens)

In [ ]:
def build_vocab(tokenized_sentences):
    counter = Counter()
    for sentence in tokenized_sentences:
        counter.update(sentence)
    v = vocab(counter, specials=['<unk>', '<pad>', '<bos>', '<eos>'], min_freq=1)
    return v

# Build source and target vocabularies
source_vocab = build_vocab(tokenized_source_sentences)
target_vocab = build_vocab(tokenized_target_sentences)

In [ ]:
print(len(source_vocab))

137163


In [ ]:
def data_process(tokenized_source, tokenized_target, source_vocab, target_vocab):
    data = []
    for (tokenized_source_sentence, tokenized_target_sentence) in zip(tokenized_source, tokenized_target):
        source_tensor = torch.tensor([source_vocab[token] for token in tokenized_source_sentence],
                                     dtype=torch.long)
        target_tensor = torch.tensor([target_vocab[token] for token in tokenized_target_sentence],
                                     dtype=torch.long)
        data.append((source_tensor, target_tensor))
    return data

# Process the data
train_data = data_process(tokenized_source_sentences, tokenized_target_sentences, source_vocab, target_vocab)

In [ ]:
batch_size = 32
pad_idx = source_vocab['<pad>']
bos_idx = source_vocab['<bos>']
eos_idx = source_vocab['<eos>']

def generate_batch(data_batch):
  source_batch, target_batch = [], []
  for (source_item, target_item) in data_batch:
    source_batch.append(torch.cat([torch.tensor([bos_idx]), source_item, torch.tensor([eos_idx])], dim=0))
    target_batch.append(torch.cat([torch.tensor([bos_idx]), target_item, torch.tensor([eos_idx])], dim=0))
  source_batch = pad_sequence(source_batch, padding_value=pad_idx)
  target_batch = pad_sequence(target_batch, padding_value=pad_idx)
  return source_batch, target_batch

train_iter = DataLoader(train_data, batch_size=batch_size,shuffle=True, collate_fn=generate_batch)

In [ ]:
class Seq2SeqTransformer(nn.Module):
    def __init__(self, num_encoder_layers: int, num_decoder_layers: int, emb_size: int, src_vocab_size: int, tgt_vocab_size: int, dim_feedforward:int = 512, dropout:float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        encoder_layer = TransformerEncoderLayer(d_model=emb_size, nhead=nhead,
                                                dim_feedforward=dim_feedforward)
        self.transformer_encoder = TransformerEncoder(encoder_layer, num_layers=num_encoder_layers)
        decoder_layer = TransformerDecoderLayer(d_model=emb_size, nhead=nhead,
                                                dim_feedforward=dim_feedforward)
        self.transformer_decoder = TransformerDecoder(decoder_layer, num_layers=num_decoder_layers)

        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(emb_size, dropout=dropout)

    def forward(self, src: Tensor, trg: Tensor, src_mask: Tensor, tgt_mask: Tensor, src_padding_mask: Tensor, tgt_padding_mask: Tensor, memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        memory = self.transformer_encoder(src_emb, src_mask, src_padding_mask)
        outs = self.transformer_decoder(tgt_emb, memory, tgt_mask, None, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer_encoder(self.positional_encoding(self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer_decoder(self.positional_encoding(self.tgt_tok_emb(tgt)), memory, tgt_mask)

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, emb_size: int, dropout, maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2) * math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0),:])

class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size
    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

In [ ]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

def create_mask(src, tgt):
  src_seq_len = src.shape[0]
  tgt_seq_len = tgt.shape[0]

  tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
  src_mask = torch.zeros((src_seq_len, src_seq_len), device=DEVICE).type(torch.bool)

  src_padding_mask = (src == pad_idx).transpose(0, 1)
  tgt_padding_mask = (tgt == pad_idx).transpose(0, 1)
  return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [ ]:
src_vocab_size = len(source_vocab)
tgt_vocab_size = len(target_vocab)
emb_size = 512
nhead = 8
ffn_hid_dim = 512
num_encoder_layers = 3
num_decoder_layers = 3
num_epochs = 40
transformer = Seq2SeqTransformer(num_encoder_layers, num_decoder_layers, emb_size, src_vocab_size, tgt_vocab_size, ffn_hid_dim)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(device)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=pad_idx)

optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [ ]:
def train_epoch(model, train_iter, optimizer):
  model.train()
  losses = 0
  for idx, (src, tgt) in enumerate(train_iter):
      src = src.to(device)
      tgt = tgt.to(device)
      tgt_input = tgt[:-1, :]
      src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)
      logits = model(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, src_padding_mask)
      optimizer.zero_grad()
      tgt_out = tgt[1:,:]
      loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
      loss.backward()
      optimizer.step()
      losses += loss.item()
  return losses / len(train_iter)

In [ ]:
for epoch in range(1, num_epochs+1):
  start_time = time.time()
  train_loss = train_epoch(transformer, train_iter, optimizer)
  end_time = time.time()
  print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))

/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:5076: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Epoch: 1, Train loss: 6.657, Epoch time = 205.889s
Epoch: 2, Train loss: 5.889, Epoch time = 206.912s
Epoch: 3, Train loss: 5.515, Epoch time = 206.351s
Epoch: 4, Train loss: 5.185, Epoch time = 207.349s
Epoch: 5, Train loss: 4.876, Epoch time = 208.018s
Epoch: 6, Train loss: 4.588, Epoch time = 207.850s
Epoch: 7, Train loss: 4.321, Epoch time = 208.101s
Epoch: 8, Train loss: 4.071, Epoch time = 207.962s
Epoch: 9, Train loss: 3.838, Epoch time = 208.046s
Epoch: 10, Train loss: 3.617, Epoch time = 207.940s
Epoch: 11, Train loss: 3.412, Epoch time = 208.090s
Epoch: 12, Train loss: 3.219, Epoch time = 207.291s
Epoch: 13, Train loss: 3.036, Epoch time = 207.001s
Epoch: 14, Train loss: 2.869, Epoch time = 207.297s
Epoch: 15, Train loss: 2.711, Epoch time = 207.359s
Epoch: 16, Train loss: 2.567, Epoch time = 206.809s
Epoch: 17, Train loss: 2.432, Epoch time = 207.031s
Epoch: 18, Train loss: 2.317, Epoch time = 206.355s
Epoch: 19, Train loss: 2.228, Epoch time = 207.458s
Epoch: 20, Train loss

KeyboardInterrupt: ignored

In [ ]:
torch.save(transformer.state_dict(), 'inference_model')

In [ ]:
def greedy_decode(model, src, src_mask, max_len, start_symbol, device):
    src = src.to(device)
    src_mask = src_mask.to(device)
    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(device)
    for i in range(max_len-1):
        memory = memory.to(device)
        memory_mask = torch.zeros(ys.shape[0], memory.shape[0]).to(device).type(torch.bool)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0)).type(torch.bool)).to(device)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()
        ys = torch.cat([ys, torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == eos_idx:
            break
    return ys

# def translate(model, src, src_vocab, tgt_vocab, device):
#     model.eval()
#     tokens = [bos_idx] + [src_vocab.get_stoi()[tok] for tok in src] + [eos_idx]
#     num_tokens = len(tokens)
#     src = (torch.LongTensor(tokens).reshape(num_tokens, 1)).to(device)
#     src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool).to(device)
#     tgt_tokens = greedy_decode(model, src, src_mask, max_len=num_tokens + 5, start_symbol=bos_idx, device=device).flatten()
#     translated_sentence= " ".join([tgt_vocab.get_itos()[tok] for tok in tgt_tokens]).replace("<bos>", "").replace("<eos>", "")
#     translated_sentence = " ".join(translated_sentence.split())
#     return translated_sentence

In [ ]:
def translate(model, src, src_vocab, tgt_vocab, device):
    model.eval()
    unk_token_idx = src_vocab.get_stoi()['<unk>']

    src_indices = [src_vocab[token] if token in src_vocab else unk_token_idx for token in src]

    tokens = [bos_idx] + src_indices + [eos_idx]
    num_tokens = len(tokens)
    src = torch.LongTensor(tokens).reshape(num_tokens, 1).to(device)

    src_mask = torch.zeros(num_tokens, num_tokens).to(device, dtype=torch.bool)
    tgt_tokens = greedy_decode(model, src, src_mask, max_len=num_tokens + 5, start_symbol=bos_idx, device=device).flatten()

    translated_tokens = [tgt_vocab.get_itos()[idx] if idx != unk_token_idx else '<unk>' for idx in tgt_tokens]

    translated_sentence = " ".join(translated_tokens).replace("<bos>", "").replace("<eos>", "")
    translated_sentence = " ".join(translated_sentence.split())
    return translated_sentence

In [ ]:
print(tokenized_source_sentences[3])

['ആദ്യത്തേതില്\u200d', 'നിന്ന്', 'ചെറുതായ', 'ചകൂട്ടില്\u200d', 'നാലുവശത്തുമായി', '12', 'വാതിലുകള്\u200d', 'ഉണ്ട്']


In [ ]:
# Create an instance of the model
loaded_transformer = Seq2SeqTransformer(num_encoder_layers, num_decoder_layers,
                                       emb_size, src_vocab_size, tgt_vocab_size,
                                       ffn_hid_dim)

# Load the saved state dictionary
loaded_transformer.load_state_dict(torch.load('inference_model'))

# Ensure the model is in evaluation mode
loaded_transformer.eval()

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Seq2SeqTransformer(
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-2): 3 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
        )
        (linear1): Linear(in_features=512, out_features=512, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=512, out_features=512, bias=True)
        (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (transformer_decoder): TransformerDecoder(
    (layers): ModuleList(
      (0-2): 3 x TransformerDecoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
        )

In [ ]:
print(len(tok_val))

15445


In [ ]:
english_pred = []
for sentence in tok_val:
  pred = translate(transformer, sentence, source_vocab, target_vocab, DEVICE)
  print(pred)
  english_pred.append(pred)

Streaming output truncated to the last 5000 lines.
after six days mumbai was hurt by over so on 23 february 2008 he died in
bandy was first held in 2019 when the bandy games was held in
if you ’re like a check online to make a language country ’s already have trees and importance might make sure you ’re willing
its cost would be more than 3000 rupees
bhojpur is situated on the road of bhojpur crossing
did i remember to set a reminder alarm for my next doctors appointment
there is a start and several start the countrys attack of the country
there are no such growth in the heat term
the face of the cold will be played in the
on 1 december nagaland became the state of nagaland was set in india
the cost of hotels has much significance of the market
അവന്‍ and the of not only are time in and
in the cold paper was signed for the mixing of muslim so far in order to hit the of a muslim so
the architecture of the of which not only is called year old
the first flying club in rajasthan is organise

In [ ]:
import csv

# Create a list of dictionaries where each dictionary represents a row
data = [{'valid_id': valid_id[i], 'english_pred': english_pred[i]} for i in range(len(valid_id))]

# Specify the CSV file path
csv_file_path = '/gdrive/MyDrive/phase2_mal2.csv'

# Define the column names
fields = ['valid_id', 'english_pred']

# Write the data to the CSV file
with open(csv_file_path, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fields)
    writer.writeheader()
    writer.writerows(data)

print(f'Saved predictions to {csv_file_path}')

Saved predictions to /gdrive/MyDrive/phase2_mal2.csv
